# V2: SEQ2SEQ

This notebook roughly follows the ["Nural Machine Translation With Attention"](https://www.tensorflow.org/text/tutorials/nmt_with_attention) tutorial. However, like the v1 notebook, it solves a different problem.

In [1]:
import numpy as np

import typing
import einops
import matplotlib.pyplot as plt

import tensorflow as tf

2024-08-31 22:15:46.893308: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-31 22:15:46.896795: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-31 22:15:46.906935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 22:15:46.922615: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 22:15:46.926957: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-31 22:15:46.938697: I tensorflow/core/platform/cpu_feature_gu

## Data setup and preprocessing

Our goal is to repunctuate a stream of text. Our input data (context) will be unpunctuated text. Our output data (target) will be the text with punctuation

In [2]:
from prepare_data import load_data

context_raw, target_raw = load_data('./data/en/', 2)
context_raw[-100:]

array([' ', 'f', 'r', 'i', 'e', 'n', 'd', 's', ' ', 'p', 'r', 'i', 'n',
       'c', 'i', 'p', 'a', 'l', 'l', 'y', ' ', 'i', 'n', ' ', 'v', 'i',
       'e', 'w', ' ', 't', 'h', 'e', 'y', ' ', 'h', 'a', 'v', 'e', ' ',
       'b', 'e', 'e', 'n', ' ', 'a', 'n', ' ', 'e', 's', 's', 'e', 'n',
       't', 'i', 'a', 'l', ' ', 'i', 'n', 'g', 'r', 'e', 'd', 'i', 'e',
       'n', 't', ' ', 'o', 'f', ' ', 'c', 'h', 'i', 'l', 'd', 'r', 'e',
       'n', 's', ' ', 'r', 'e', 'a', 'd', 'i', 'n', 'g', ' ', 'e', 'v',
       'e', 'r', ' ', 's', 'i', 'n', 'c', 'e', ' '], dtype='<U1')

In [4]:
target_raw[-100:]

array(['e', 'n', 'd', 's', ' ', 'p', 'r', 'i', 'n', 'c', 'i', 'p', 'a',
       'l', 'l', 'y', ' ', 'i', 'n', ' ', 'v', 'i', 'e', 'w', '.', "'",
       ' ', 'T', 'h', 'e', 'y', ' ', 'h', 'a', 'v', 'e', ' ', 'b', 'e',
       'e', 'n', ' ', 'a', 'n', ' ', 'e', 's', 's', 'e', 'n', 't', 'i',
       'a', 'l', ' ', 'i', 'n', 'g', 'r', 'e', 'd', 'i', 'e', 'n', 't',
       ' ', 'o', 'f', ' ', 'c', 'h', 'i', 'l', 'd', 'r', 'e', 'n', "'",
       's', ' ', 'r', 'e', 'a', 'd', 'i', 'n', 'g', ' ', 'e', 'v', 'e',
       'r', ' ', 's', 'i', 'n', 'c', 'e', '.', ' '], dtype='<U1')

## The Encoder

As explained in [the tutorial](https://www.tensorflow.org/text/tutorials/nmt_with_attention#the_attention_layer), the encoder's job is to build a new hidden sequence from the provided list of tokens.

In [ ]:
class Encoder(tf.keras.layers.Layer):
	def __init__(self, text_processor, units)